In [1]:
from data_preprocessing import loadData,getScaledAndOneHotEncoderedX,getLineFromFile,simpleScale,testModelOnData,scaleWithFeaturesAndKeepLocation
from data_preprocessing import checkNegative
from data_preprocessing import decisionTreeDemo

In [32]:
def randomForestDemo(X, y, TX, Ty):
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier()
    testModelOnData(model, X, y, TX, Ty)


In [3]:
def extraTreeDemo(X, y, TX, Ty):
    from sklearn.ensemble import ExtraTreesClassifier
    # model = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
    model = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2,
                                 max_features='sqrt', random_state=0)
    testModelOnData(model, X, y, TX, Ty)

In [4]:
X, y, TX, Ty = loadData()

scaledAndOneHotX = getScaledAndOneHotEncoderedX(X) 
scaledAndOneHotTX = getScaledAndOneHotEncoderedX(TX) 
print(scaledAndOneHotX.shape)

32561 15
16281 15
(32561, 105)


In [5]:
continuous_features = list(map(lambda x: x - 1, [1, 3, 5, 11, 12, 13]))
scaledX = scaleWithFeaturesAndKeepLocation(X, continuous_features)
scaledTX = scaleWithFeaturesAndKeepLocation(TX, continuous_features)
print(X.shape, scaledX.shape)

X12 = X[:3]
scaledX12 = scaledX[:3]
print(X12[:,continuous_features])
print(scaledX12[:,continuous_features])


(32561, 14) (32561, 14)
[[  3.90000000e+01   7.75160000e+04   1.30000000e+01   2.17400000e+03
    0.00000000e+00   4.00000000e+01]
 [  5.00000000e+01   8.33110000e+04   1.30000000e+01   0.00000000e+00
    0.00000000e+00   1.30000000e+01]
 [  3.80000000e+01   2.15646000e+05   9.00000000e+00   0.00000000e+00
    0.00000000e+00   4.00000000e+01]]
[[ 0.03067056 -1.06361075  1.13473876  0.1484529  -0.21665953 -0.03542945]
 [ 0.83710898 -1.008707    1.13473876 -0.14592048 -0.21665953 -2.22215312]
 [-0.04264203  0.2450785  -0.42005962 -0.14592048 -0.21665953 -0.03542945]]


In [6]:
'''
[All decision trees use np.float32 arrays internally. If training data is not in this format, a copy of the dataset will be made.](http://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use)
Do not see difference.
'''

import numpy as np
X = X.astype(np.float32)
TX = TX.astype(np.float32)
scaledX = scaledX.astype(np.float32)
scaledTX = scaledTX.astype(np.float32)
scaledAndOneHotX = scaledAndOneHotX.astype(np.float32)
scaledAndOneHotTX = scaledAndOneHotTX.astype(np.float32)





In [7]:
feature_names = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country']
class_names = ["gt 50K","le 50K"]
print(len(feature_names))

14


In [8]:
'''
Usage:
    saveTree2DotAndPdf(clf, feature_names=feature_names, class_names=class_names)
'''
def saveTree2DotAndPdf(clf, feature_names, class_names, file_name='tmp', showImage=False):
    from sklearn import tree
    dotFile = file_name+".dot"
    pdfFile = file_name+".pdf"
    with open(dotFile, 'w') as f:
        f = tree.export_graphviz(clf, out_file=f,
                            feature_names=feature_names,
                            class_names=class_names,
                            filled=True, 
                            rounded=True,  
                            special_characters=True
                            )
    print('save to dot file done\n')
    import pydotplus 
    # dot_data = tree.export_graphviz(clf, out_file=None) 
    # graph = pydotplus.graph_from_dot_data(dot_data) 
    graph = pydotplus.graph_from_dot_file(dotFile)
    graph.write_pdf(pdfFile)
    print('save to pdf file done\n')

    
    if showImage:
        print('prepring to show the image, may take a long time...\n')
        from IPython.display import Image
        Image(graph.create_png())
        
        

In [9]:
# Default decisionTree
decisionTreeDemo(X, y, TX, Ty)
decisionTreeDemo(scaledX, y, scaledTX, Ty)
decisionTreeDemo(scaledAndOneHotX,y,scaledAndOneHotTX,Ty)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      7841
        1.0       1.00      1.00      1.00     24720

avg / total       1.00      1.00      1.00     32561

[[ 7841     0]
 [    1 24719]]


             precision    recall  f1-score   support

        0.0       0.60      0.62      0.61      3846
        1.0       0.88      0.87      0.88     12435

avg / total       0.82      0.81      0.81     16281

[[ 2372  1474]
 [ 1562 10873]]


[ 0.80818132  0.8133407   0.81258638]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_s

In [10]:
# [1.10.5. Tips on practical use](http://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use)

#1 feature selection/dimensionality reduction 

'''
## 1.1 SelectKBest
Test different k,found when k=2, we got best scores.
The parameters selected is [10,11], aka, 'capital-gain' and 'capital-loss'
'''

from sklearn.feature_selection import SelectKBest,chi2
skb = SelectKBest(chi2, k=2).fit(X, y) 

X_new = skb.transform(X)
TX_new = skb.transform(TX)


print(X.shape)
print(X_new.shape,TX_new.shape)
print(skb.get_support(indices=True))

decisionTreeDemo(X_new, y, TX_new, Ty)



(32561, 14)
(32561, 2) (16281, 2)
[10 11]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
             precision    recall  f1-score   support

        0.0       0.97      0.31      0.47      7841
        1.0       0.82      1.00      0.90     24720

avg / total       0.86      0.83      0.80     32561

[[ 2450  5391]
 [   75 24645]]


             precision    recall  f1-score   support

        0.0       0.97      0.31      0.47      3846
        1.0       0.82      1.00      0.90     12435

avg / total       0.86      0.83      0.80     16281

[[ 1184  2662]
 [   36 12399]]


[ 0.83287267  0.83287267  0.82861882]


In [11]:
'''
# Select K best with scaled data

No imporvement
'''

from sklearn.feature_selection import SelectKBest,chi2
skb = SelectKBest(chi2, k=2).fit(X, y) 

checkNegative(scaledX)

scaledX_new = skb.transform(scaledX)
scaledTX_new = skb.transform(scaledTX)


print(X.shape)
print(scaledX_new.shape,scaledTX_new.shape)
print(skb.get_support(indices=True))



decisionTreeDemo(scaledX_new, y, scaledTX_new, Ty)



'''
chi2 failed because of it [Compute chi-squared stats between each non-negative feature and class](https://github.com/scikit-learn/scikit-learn/blob/14031f6/sklearn/feature_selection/univariate_selection.py#L305)
'''

print(scaledAndOneHotX.shape)
# checkNegative(scaledAndOneHotX)

from sklearn.feature_selection import SelectKBest,chi2,f_classif

skb2 = SelectKBest(f_classif, k=50).fit(scaledAndOneHotX, y)
scaledAndOneHotXKBest = skb2.transform(scaledAndOneHotX)
scaledAndOneHotTXKBest = skb2.transform(scaledAndOneHotTX)

decisionTreeDemo(scaledAndOneHotXKBest, y, scaledAndOneHotTXKBest, Ty)


Enter ckeck negative..
141817
(0, 2)
-1.06361
(32561, 14)
(32561, 2) (16281, 2)
[10 11]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
             precision    recall  f1-score   support

        0.0       0.97      0.31      0.47      7841
        1.0       0.82      1.00      0.90     24720

avg / total       0.86      0.83      0.80     32561

[[ 2450  5391]
 [   75 24645]]


             precision    recall  f1-score   support

        0.0       0.92      0.24      0.39      3846
        1.0       0.81      0.99      0.89     12435

avg / total       0.84      0.82      0.77     16281

[[  939  2907]
 [   84 12351]]


[ 0.83287267  0.83287267  0.82861882]
(32561, 105)
DecisionTreeClassifier(class_weight=None, criterion='

In [30]:
'''
## 1.2 Tree-based feature selection

It seems that we found nice attrs to use, but the default tree is not so good.
'''

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier()
clf = clf.fit(X, y)
print(clf.feature_importances_)

model = SelectFromModel(clf, prefit=True)
extraTreesX = model.transform(X)
extraTreesTX = model.transform(TX)
print(extraTreesX.shape, extraTreesTX.shape)
print(extraTreesX[1])

decisionTreeDemo(extraTreesX, y, extraTreesTX, Ty)



# scaled data
clf = clf.fit(X, y)
print(clf.feature_importances_)

model = SelectFromModel(clf, prefit=True)
extraTreesScaledX = model.transform(scaledX)
extraTreesScaledTX = model.transform(scaledTX)
print(scaledX.shape, scaledTX.shape, extraTreesScaledX.shape, extraTreesScaledTX.shape)
print(extraTreesScaledX[1])

decisionTreeDemo(extraTreesScaledX, y, extraTreesScaledTX, Ty)



# scaled and onehot encodered data
clf.fit(scaledAndOneHotX, y)
model = SelectFromModel(clf, prefit=True)
extraTreeScaledAndOneHotX = model.transform(scaledAndOneHotX)
extraTreeScaledAndOneHotTX = model.transform(scaledAndOneHotTX)
print(scaledAndOneHotX.shape, scaledAndOneHotTX.shape, extraTreeScaledAndOneHotX.shape, extraTreeScaledAndOneHotTX.shape)

decisionTreeDemo(extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty)

[ 0.15265516  0.04189486  0.16356164  0.03056916  0.08820702  0.08197178
  0.07991579  0.0711433   0.01323944  0.03709689  0.09349672  0.03161799
  0.09370832  0.02092191]
(32561, 7) (16281, 7)
[  5.00000000e+01   8.33110000e+04   1.30000000e+01   0.00000000e+00
   4.00000000e+00   0.00000000e+00   1.30000000e+01]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      7841
        1.0       1.00      1.00      1.00     24720

avg / total       1.00      1.00      1.00     32561

[[ 7841     0]
 [    4 24716]]


             precision    recall  f1-score   support

        0.0       0.58      0.59      0.58      3846
        1.0      

In [28]:
'''
## 1.3 PCA way

Use the extraTree got nums as the n_components, aka, X->7, scalaedX->7, scaledAndOneHotX->19

params:
    >batch_size,The number of samples to use for each batch. 
     If batch_size is None, then batch_size is inferred from the data and set to 5 * n_features. 
     
     Not suitable for our onehot encodered data.

'''

from sklearn.decomposition import PCA, IncrementalPCA

n_components = 7
ipca = IncrementalPCA(n_components=n_components, batch_size=None)
ipca.fit(X, y)
pcaX = ipca.transform(X)
pcaTX = ipca.transform(TX)

print(pcaX.shape, pcaTX.shape)

n_components = 7
ipca = IncrementalPCA(n_components=n_components, batch_size=None)
pcaScaledX = ipca.fit_transform(scaledX)
pcaScaledTX = ipca.fit_transform(scaledTX)

print(pcaScaledX.shape, pcaScaledTX.shape)


n_components = 19
ipca2 = IncrementalPCA(n_components=n_components, batch_size=50)
pcaScaledAndOneHotX = ipca2.fit_transform(scaledAndOneHotX)
pcaScaledAndOneHotTX = ipca2.fit_transform(scaledAndOneHotTX)

print(pcaScaledAndOneHotX.shape, pcaScaledAndOneHotTX.shape)





(32561, 7) (16281, 7)
(32561, 7) (16281, 7)
(32561, 19) (16281, 19)


In [81]:
'''
Tweak the params to get better performance by hand.

1.Only change the max_depth, [5-10] gives similar nice results of 0.84, and train/test datasets seems alike.

2.[Balance your dataset before training to prevent the tree from being biased toward the classes that are dominant.](http://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use)
class_weight="balanced", see [class_weight : dict, list of dicts, “balanced” or None, optional (default=None)](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
     However, not so appealling.

3.
'''

from sklearn import tree
max_features = ceil(sqrt(X.shape[1]))
print(max_features)

model = tree.DecisionTreeClassifier(max_depth=10,max_features='sqrt')
testModelOnData(model, X, y, TX, Ty)
testModelOnData(model, extraTreesX, y, extraTreesTX, Ty)
testModelOnData(model, extraTreesScaledX, y, extraTreesScaledTX, Ty)
# testModelOnData(model, extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty)

# testModelOnData(model, pcaX, y, pcaTX, Ty)
# testModelOnData(model, pcaScaledX, y, pcaScaledTX, Ty)
# testModelOnData(model, pcaScaledAndOneHotX, y, pcaScaledAndOneHotTX, Ty)


# saveTree2DotAndPdf(model, feature_names=feature_names, class_names=class_names, file_name='maxdepth4_extraTree')

4
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
             precision    recall  f1-score   support

        0.0       0.80      0.56      0.66      7841
        1.0       0.87      0.96      0.91     24720

avg / total       0.86      0.86      0.85     32561

[[ 4357  3484]
 [ 1071 23649]]


             precision    recall  f1-score   support

        0.0       0.76      0.52      0.62      3846
        1.0       0.87      0.95      0.90     12435

avg / total       0.84      0.85      0.84     16281

[[ 2013  1833]
 [  644 11791]]


[ 0.8398747   0.84457343  0.84594121]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features='sqrt', max_leaf_nodes=None,
            min_impurity

In [82]:

'''
Tweak RandomForest params to get better performance by hand.
[1.11.2.3. Parameters](http://scikit-learn.org/stable/modules/ensemble.html#parameters)

1.n_estimators
    the number of trees in the forest. The larger the better, but also the longer it will take to compute. 

2.max_features    
    max_features=sqrt(n_features) for classification tasks

[3.2.4.3.1. sklearn.ensemble.RandomForestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)

3.oob_score
    Whether to use out-of-bag samples to estimate the generalization accuracy.

4.n_jobs
    If -1, then the number of jobs is set to the number of cores.
    
5.random_state
    const for repeat results
'''

from sklearn.ensemble import RandomForestClassifier
from math import sqrt,ceil

'''
# baseline

They seems nice, but overfit, should have better results.
'''
# randomForestDemo(X, y, TX, Ty) # [ 0.84374424  0.84208587  0.84630978]
# randomForestDemo(extraTreesX, y, extraTreesTX, Ty) # [ 0.83176709  0.83407039  0.83654289]
# randomForestDemo(extraTreesScaledX, y, extraTreesScaledTX, Ty) # [ 0.83103004  0.84051962  0.83165945]
# randomForestDemo(extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty) # [ 0.82881887  0.83250415  0.83433152]

max_features = ceil(sqrt(X.shape[1]))
print(max_features)
model = RandomForestClassifier(n_estimators=50, max_depth=10,max_features='sqrt',oob_score=True,#lass_weight='balanced',
    min_samples_split=2, random_state=0,n_jobs=-1)

'''
Results report, orginal data and extraTreeScaledAndOneHotX:

[50,10,max_features]
    [ 0.85535287  0.86060439  0.86114438]
    [ 0.85562926  0.86051225  0.8609601 ]

[50,10,max_features,oob_score]
    [ 0.85535287  0.86060439  0.86114438]
    [ 0.8551686   0.86189423  0.86197365]
    
class_weight='balanced' -> still worse
    [ 0.80974756  0.80615441  0.81258638]
    [ 0.80864198  0.80421964  0.8128628 ]

'''


# testModelOnData(model, X, y, TX, Ty)
# testModelOnData(model, extraTreesX, y, extraTreesTX, Ty)
# testModelOnData(model, extraTreesScaledX, y, extraTreesScaledTX, Ty)


max_features2 = ceil(sqrt(extraTreeScaledAndOneHotX.shape[1]))
print(extraTreeScaledAndOneHotX.shape,max_features2)
testModelOnData(model, extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty)





'''
The pca results are all 0.83**, not so appealing
'''
print(pcaX.shape, pcaScaledX.shape, pcaScaledAndOneHotX.shape)

# testModelOnData(model, pcaX, y, pcaTX, Ty)
# testModelOnData(model, pcaScaledX, y, pcaScaledTX, Ty)
# testModelOnData(model2, pcaScaledAndOneHotX, y, pcaScaledAndOneHotTX, Ty)


# saveTree2DotAndPdf(model, feature_names=feature_names, class_names=class_names, file_name='maxdepth4_extraTree')

4
(32561, 19) 5
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=-1, oob_score=True, random_state=0,
            verbose=0, warm_start=False)
             precision    recall  f1-score   support

        0.0       0.82      0.58      0.68      7841
        1.0       0.88      0.96      0.92     24720

avg / total       0.86      0.87      0.86     32561

[[ 4561  3280]
 [ 1003 23717]]


             precision    recall  f1-score   support

        0.0       0.78      0.54      0.64      3846
        1.0       0.87      0.95      0.91     12435

avg / total       0.85      0.86      0.85     16281

[[ 2093  1753]
 [  576 11859]]


[ 0.85507647  0.86051225  0.86059154]
(32561, 7) (32561, 7) (32561, 19)


In [ ]:
'''
Tuning RandomForest by GridSearchCV, this should get better results than hand chosen.

[3.2. Tuning the hyper-parameters of an estimator](http://scikit-learn.org/stable/modules/grid_search.html#tuning-the-hyper-parameters-of-an-estimator)


The GridSearchCV instance implements the usual estimator API: when “fitting” it on a dataset all the possible combinations of parameter values are evaluated and the best combination is retained.

'''





from sklearn.ensemble import RandomForestClassifier


# model = RandomForestClassifier(n_estimators=50, max_depth=10,max_features='sqrt',oob_score=True,#lass_weight='balanced',
#     min_samples_split=2, random_state=0,n_jobs=-1)

model = RandomForestClassifier()

'''
model.get_params()

{'min_samples_leaf': 1, 'max_features': 'auto', 'n_jobs': 1, 'class_weight': None, 'bootstrap': True, 'warm_start': False,
    'min_weight_fraction_leaf': 0.0, 'random_state': None, 'verbose': 0, 'criterion': 'gini', 'oob_score': False, 
    'min_impurity_split': 1e-07, 'min_samples_split': 2, 'n_estimators': 10, 'max_depth': None, 'max_leaf_nodes': None}
'''



from sklearn.model_selection import GridSearchCV


params = {'n_estimators':[50,70,90,110], 'max_features':['sqrt','log2',None],'oob_score':[True,False],'max_depth':[5,7,9,11,13],
         'min_samples_split':[2,3,4]}

grid = GridSearchCV(estimator=model, param_grid=params, cv=5,
                   n_jobs=-1)

testModelOnData(grid, X, y, TX, Ty)

'''
Failed to get all results


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [50, 70, 90, 110], 'oob_score': [True, False], 'max_depth': [5, 7, 9, 11, 13], 'min_samples_split': [2, 3, 4], 'max_features': ['sqrt', 'log2', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)
             precision    recall  f1-score   support

        0.0       0.83      0.65      0.73      7841
        1.0       0.90      0.96      0.93     24720

avg / total       0.88      0.88      0.88     32561

[[ 5070  2771]
 [ 1047 23673]]


             precision    recall  f1-score   support

        0.0       0.78      0.60      0.67      3846
        1.0       0.88      0.95      0.91     12435

avg / total       0.86      0.86      0.86     16281

[[ 2292  1554]
 [  654 11781]]
'''


# testModelOnData(model, extraTreesX, y, extraTreesTX, Ty)
# testModelOnData(model, extraTreesScaledX, y, extraTreesScaledTX, Ty)
# testModelOnData(model, extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty)







GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [50, 70, 90, 110], 'oob_score': [True, False], 'max_depth': [5, 7, 9, 11, 13], 'min_samples_split': [2, 3, 4], 'max_features': ['sqrt', 'log2', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)
             precision    recall  f1-score   support

        0.0       0.83      0.65      0.73      7841
        1.0       0.90      0.96      0.93     24720

avg / total       0.88      0.88      0.88     32561

[[ 5070  2771

In [ ]:
'''
http://scikit-learn.org/stable/modules/grid_search.html#specifying-an-objective-metric
     For some applications, other scoring functions are better suited (for example in unbalanced classification, the accuracy score is often uninformative).

So we use `f1` for alternative


refit : boolean, default=True

    Refit the best estimator with the entire dataset. If “False”, it is impossible to make predictions using this GridSearchCV instance after fitting.

'''

from sklearn.ensemble import RandomForestClassifier


# model = RandomForestClassifier(n_estimators=50, max_depth=10,max_features='sqrt',oob_score=True,#lass_weight='balanced',
#     min_samples_split=2, random_state=0,n_jobs=-1)

model = RandomForestClassifier()

'''
model.get_params()

{'min_samples_leaf': 1, 'max_features': 'auto', 'n_jobs': 1, 'class_weight': None, 'bootstrap': True, 'warm_start': False,
    'min_weight_fraction_leaf': 0.0, 'random_state': None, 'verbose': 0, 'criterion': 'gini', 'oob_score': False, 
    'min_impurity_split': 1e-07, 'min_samples_split': 2, 'n_estimators': 10, 'max_depth': None, 'max_leaf_nodes': None}
'''



from sklearn.model_selection import GridSearchCV


params = {'n_estimators':[50,110], 'max_features':['sqrt','log2'],'oob_score':[True],'max_depth':[5,9,11],
         'min_samples_split':[2,4]}

grid = GridSearchCV(estimator=model, param_grid=params, cv=5,
                    scoring='f1', #see above
                    error_score=0, # to avoid crash
                   n_jobs=-1)

testModelOnData(grid, X, y, TX, Ty)
# testModelOnData(model, extraTreesX, y, extraTreesTX, Ty)
# testModelOnData(model, extraTreesScaledX, y, extraTreesScaledTX, Ty)
# testModelOnData(model, extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty)

In [79]:
'''
We should think the best results can be 
    For learning rate 0.1
        n_estimators=10000
        
        [ 0.86668509  0.8684356   0.87275408]
'''


'''
Adaboosting TRY


>An AdaBoost [1] classifier is a meta-estimator that begins by fitting a classifier on the original dataset and then fits additional 
copies of the classifier on the same dataset but where the weights of incorrectly classified instances are adjusted such that subsequent 
classifiers focus more on difficult cases
--http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier



It seems nice, sine the defalt have 
[ 0.85701124  0.85866961  0.86252649] on orginal data.

params:
1.learning rate
    Learning rate shrinks the contribution of each classifier by learning_rate. There is a trade-off between learning_rate and n_estimators
    
    For learning rate 0.5,
        n_estimators=500
        [ 0.8641054   0.86825134  0.86980558]
        
        1000
        [ 0.86557951  0.86797494  0.87229338]
    
    For learning rate 0.1
        n_estimators=10000
        
        [ 0.86668509  0.8684356   0.87275408]        

'''

from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier


model = AdaBoostClassifier(n_estimators=10000, learning_rate=0.1,
                           random_state=0)

testModelOnData(model, X, y, TX, Ty)
# testModelOnData(model, extraTreesX, y, extraTreesTX, Ty)
# testModelOnData(model, extraTreesScaledX, y, extraTreesScaledTX, Ty)
# testModelOnData(model, extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.1, n_estimators=10000, random_state=0)
             precision    recall  f1-score   support

        0.0       0.79      0.64      0.71      7841
        1.0       0.89      0.95      0.92     24720

avg / total       0.87      0.87      0.87     32561

[[ 5027  2814]
 [ 1330 23390]]


             precision    recall  f1-score   support

        0.0       0.77      0.64      0.70      3846
        1.0       0.89      0.94      0.92     12435

avg / total       0.87      0.87      0.87     16281

[[ 2444  1402]
 [  710 11725]]


[ 0.86668509  0.8684356   0.87275408]


In [85]:
'''
GradientBoostingClassifier



parmas:
    n_estimators : int (default=100)
    The number of boosting stages to perform. Gradient boosting is **fairly robust to over-fitting** so a large number usually results in better performance.

'''

from sklearn.ensemble import GradientBoostingClassifier


model = GradientBoostingClassifier(n_estimators=2000, 
                                   learning_rate=0.1,
                                   max_features='sqrt',
#                                     max_depth=1,
                                   random_state=0)


testModelOnData(model, X, y, TX, Ty)
# testModelOnData(model, extraTreesX, y, extraTreesTX, Ty)
# testModelOnData(model, extraTreesScaledX, y, extraTreesScaledTX, Ty)
# testModelOnData(model, extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=2000, presort='auto', random_state=0,
              subsample=1.0, verbose=0, warm_start=False)
             precision    recall  f1-score   support

        0.0       0.84      0.72      0.77      7841
        1.0       0.91      0.96      0.93     24720

avg / total       0.90      0.90      0.90     32561

[[ 5631  2210]
 [ 1104 23616]]


             precision    recall  f1-score   support

        0.0       0.77      0.66      0.71      3846
        1.0       0.90      0.94      0.92     12435

avg / total       0.87      0.87      0.87     16281

[[ 2533  1313]
 [  761 11674]]


[ 0.86917266  0.86760641  0.86851562]


In [92]:
from data_preprocessing import logisticRegression,kNeighborsDemo,cross_validation

In [103]:
'''
Combine method

And tuning.

'''
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = KNeighborsClassifier()

model = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('knn', clf3)], voting='hard')

'''
[ 0.79767828  0.79887599  0.79839676]
[ 0.84116455  0.84798231  0.84769188]
[ 0.77833057  0.7740925   0.7734267 ]
[ 0.81730238  0.81776304  0.81793053]
'''
# for clf, label in zip([clf1, clf2, clf3, model], ['Logistic Regression', 'Random Forest', 'Knn', 'Ensemble']):
#     scores = cross_validation(clf, X, y)
#     print(scores)
    

    
'''

'''        
# for clf, label in zip([clf1, clf2, clf3, model], ['Logistic Regression', 'Random Forest', 'Knn', 'Ensemble']):
#     scores = cross_validation(clf, pcaScaledAndOneHotX, y)
#     print(scores)

# testModelOnData(model, X, y, TX, Ty)
# testModelOnData(model, extraTreesX, y, extraTreesTX, Ty)
# testModelOnData(model, extraTreesScaledX, y, extraTreesScaledTX, Ty)
# testModelOnData(model, extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty)



'''
[3.2. Tuning the hyper-parameters of an estimator](http://scikit-learn.org/stable/modules/grid_search.html#tuning-the-hyper-parameters-of-an-estimator)


'''

from sklearn.model_selection import GridSearchCV
params = {'lr__C': [1.0, 100.0], 'rf__n_estimators': [20, 200],}
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5,
                   n_jobs=-1)
testModelOnData(grid, X, y, TX, Ty)



GridSearchCV(cv=5, error_score='raise',
       estimator=VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomFore...owski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))],
         n_jobs=1, voting='hard', weights=None),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'rf__n_estimators': [20, 200], 'lr__C': [1.0, 100.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)
             precision    recall  f1-score   support

        0.0       0.94      0.49      0.65      7841
        1.0       0.86      0.99      0.92     24720

avg / total       0.88      0.87      0.85     32561

[[ 3862  3979]
 [  256 24464]]


      